# Beam conversion from Bigquery to TF Records

In this notebook we use Apache Beam to convert to tfrecords
The applications can be found in `beam_candidates` and `beam_training` for candidate generation and training

## Load env config

In [1]:
# naming convention for all cloud resources
VERSION        = "v1"                  # TODO
PREFIX         = f'ndr-{VERSION}'      # TODO

print(f"PREFIX = {PREFIX}")

PREFIX = ndr-v1


In [2]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "ndr-v1"
VERSION                  = "v1"

APP                      = "sp"
MODEL_TYPE               = "2tower"
FRAMEWORK                = "tfrs"
DATA_VERSION             = "v1"
TRACK_HISTORY            = "5"

BUCKET_NAME              = "ndr-v1-hybrid-vertex-bucket"
BUCKET_URI               = "gs://ndr-v1-hybrid-vertex-bucket"
SOURCE_BUCKET            = "spotify-million-playlist-dataset"

DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://ndr-v1-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

CANDIDATE_PREFIX         = "candidates"
TRAIN_DIR_PREFIX      

In [3]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://ndr-v1-hybrid-vertex-bucket/config/
                                 gs://ndr-v1-hybrid-vertex-bucket/data/
                                 gs://ndr-v1-hybrid-vertex-bucket/endpoint-tests/
                                 gs://ndr-v1-hybrid-vertex-bucket/local-train-v1/
                                 gs://ndr-v1-hybrid-vertex-bucket/local-train-v2/
                                 gs://ndr-v1-hybrid-vertex-bucket/scale-training-v1/
                                 gs://ndr-v1-hybrid-vertex-bucket/tfrs-pipe-v1/


## pip & package

In [4]:
# !pip install --upgrade 'apache-beam[gcp]' --user

In [5]:
import os
import time
from pprint import pprint

import logging
logging.disable(logging.WARNING)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf

from google.cloud import storage

import warnings
warnings.filterwarnings('ignore')

In [6]:
storage_client = storage.Client(project=PROJECT_ID)

# Run Dataflow to convert BQ to TFrecords

Candidate generation can be found in `beam_candidates`
Training and Validation generation can be found in `beam_training`

Usage:
* Candidate generation

> `beam_candidates\python3 main.py $PROJECT_ID $NETWORK $REGION $VERSION $BUCKET_NAME $CANDIDATE_PREFIX $BQ_DATASET $BQ_TABLE_CANDIDATES`
   
* Training generation
  
> `beam_training\python3 main-train.py <BQ_table> <gcs data subfolder> <desired partition size MB> <BQ dataset size MB> <version tag>`

In [7]:
!tree beam_training

beam_training
├── README.MD
├── __init__.py
├── create_tfrecords_training.egg-info
│   ├── PKG-INFO
│   ├── SOURCES.txt
│   ├── dependency_links.txt
│   ├── requires.txt
│   └── top_level.txt
├── main-train.py
├── setup.py
└── train_pipeline
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-37.pyc
    │   ├── train_pipe.cpython-37.pyc
    │   └── train_pipe_shape.cpython-37.pyc
    └── train_pipe_shape.py

3 directories, 14 files


In [10]:
# os.getcwd()

In [10]:
%cd beam_candidates

/home/jupyter/jw-repo2/spotify_mpd_two_tower/beam_candidates


### Candidates

In [12]:
start_time = time.time()

! python3 main.py $PROJECT_ID $VPC_NETWORK_NAME $REGION $DATA_VERSION $BUCKET_NAME $CANDIDATE_PREFIX $BQ_DATASET $BQ_TABLE_CANDIDATES

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-candidates-v3data-230919-130656, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://ndr-v1-hybrid-vertex-bucket/data/v3data/job/staging/, temp_location=gs://ndr-v1-hybrid-vertex-bucket/data/v3data/job/temp/, template_location=None, transform

In [29]:
!pwd

/home/jupyter/jw-repo2/spotify_mpd_two_tower/beam_training


In [30]:
%cd ..

/home/jupyter/jw-repo2/spotify_mpd_two_tower


### Validation set

In [25]:
%cd beam_training

/home/jupyter/jw-repo2/spotify_mpd_two_tower/beam_training


In [33]:
TARGET_SHARD_SIZE_MB_VALID = 250
TOTAL_MB_VALID = 500
NUM_TF_RECORDS = int(TOTAL_MB_VALID) // int(TARGET_SHARD_SIZE_MB_VALID)
NUM_TF_RECORDS

2

In [17]:
start_time = time.time()

# ! python3 main-train.py $BQ_TABLE_VALID $VALID_DIR_PREFIX $TARGET_SHARD_SIZE_MB_VALID $TOTAL_MB_VALID $VERSION $BUCKET_NAME $REGION $PROJECT_ID $NETWORK $BQ_DATASET

! python3 main-train.py $PROJECT_ID $VPC_NETWORK_NAME $REGION $DATA_VERSION $BUCKET_NAME $VALID_DIR_PREFIX $TOTAL_MB_VALID $TARGET_SHARD_SIZE_MB_VALID $BQ_DATASET $BQ_TABLE_VALID

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

Number of Expected TFRecords: 2
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-v1-230922-031920, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://ndr-v1-hybrid-vertex-bucket/data/v1/job/staging/, temp_location=gs://ndr-v1-hybrid-vertex-bucket/data/v1/job/temp/, template_location=None, 

### Tain set

In [26]:
TARGET_SHARD_SIZE_MB_TRAIN = 2000
TOTAL_MB_TRAIN = 44_000
NUM_TF_RECORDS = int(TOTAL_MB_TRAIN) // int(TARGET_SHARD_SIZE_MB_TRAIN)
NUM_TF_RECORDS

22

In [27]:
start_time = time.time()

! python3 main-train.py $PROJECT_ID $VPC_NETWORK_NAME $REGION $DATA_VERSION $BUCKET_NAME $TRAIN_DIR_PREFIX $TOTAL_MB_TRAIN $TARGET_SHARD_SIZE_MB_TRAIN $BQ_DATASET $BQ_TABLE_TRAIN

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

Number of Expected TFRecords: 22
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-v1-230922-104236, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://ndr-v1-hybrid-vertex-bucket/data/v1/job/staging/, temp_location=gs://ndr-v1-hybrid-vertex-bucket/data/v1/job/temp/, template_location=None,

# Test TFRecords

In [7]:
from util import feature_set_utils as feature_utils

## Candidates

### Candidate tower features

In [8]:
candidate_features = feature_utils.get_candidate_features()
candidate_features

{'track_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'track_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'duration_ms_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_genres_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_followers_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_danceability_can': FixedLenFeature(shape=(), dtype=tf.float32, defa

### Candidate files

In [18]:
candidate_files = []

for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'data/{DATA_VERSION}/{CANDIDATE_PREFIX}'):
    candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

candidate_dataset = tf.data.TFRecordDataset(candidate_files)

parsed_candidate_dataset = candidate_dataset.map(feature_utils.parse_candidate_tfrecord_fn)

In [19]:
for x in parsed_candidate_dataset.batch(1).take(1):
    pprint(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Thanatophobia'], dtype=object)>,
 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:5GBUYg5EqeDI0CuszAvDzj'], dtype=object)>,
 'artist_followers_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([27438.], dtype=float32)>,
 'artist_genres_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"'indie garage rock'"], dtype=object)>,
 'artist_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Worn-Tin'], dtype=object)>,
 'artist_pop_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.], dtype=float32)>,
 'artist_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:artist:7j8ds7BnqaEKuz1a1GN0J9'], dtype=object)>,
 'duration_ms_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([216923.], dtype=float32)>,
 'track_acousticness_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.655], dtype=float32)>,
 'track_danceability_can

## Train & Valid datasets

In [9]:
# TRACK_HISTORY

### Get all features

In [10]:
feats = feature_utils.get_all_features(TRACK_HISTORY, ranker=True)
feats

{'track_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'track_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'duration_ms_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_genres_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_followers_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_danceability_can': FixedLenFeature(shape=(), dtype=tf.float32, defa

### Choose feature set mappings

> TODO: define mappings

* `towers`
* `rank`
* `audio-rank`
* `lw-audio-rank`

```python
def get_feature_mapping(key):
    """
    returns chosen parse function
    
    example:
        desired_mapping = get_feature_mapping(MY_CHOICE)
    """
    
    map_dict = {
        "towers": feature_utils.parse_towers_tfrecord,
        "rank": feature_utils.parse_rank_tfrecord,
        "audio-rank": feature_utils.parse_audio_rank_tfrecord,
        "lw-audio-rank": feature_utils.parse_lw_audio_rank_tfrecord,
    }
    return map_dict[key]
```

### Valid files

In [11]:
FEATURE_MAP = feature_utils.parse_rank_tfrecord

In [12]:
valid_files = []

for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'data/{DATA_VERSION}/{VALID_DIR_PREFIX}/'):
    if '.tfrecords' in blob.name:
        valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
valid = tf.data.TFRecordDataset(valid_files)

print(f"valid size: {len(list(valid))}")

valid_parsed = valid.map(FEATURE_MAP)
# valid_parsed

valid size: 662124


In [13]:
for x in valid_parsed.batch(1).take(1):
    pprint(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"B'Day"], dtype=object)>,
 'album_name_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b"B'Day Deluxe Edition", b"B'Day Deluxe Edition",
        b"B'Day Deluxe Edition", b"B'Day Deluxe Edition", b"B'Day"]],
      dtype=object)>,
 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:3MJHoQUI828kmB6IpjejbW'], dtype=object)>,
 'album_uri_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b'spotify:album:2WkOVbvgyo0E0G7w2ZsAi2',
        b'spotify:album:2WkOVbvgyo0E0G7w2ZsAi2',
        b'spotify:album:2WkOVbvgyo0E0G7w2ZsAi2',
        b'spotify:album:0JLv6iVbeiy4Dh2eIw6FKI',
        b'spotify:album:3MJHoQUI828kmB6IpjejbW']], dtype=object)>,
 'artist_followers_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([33813884.], dtype=float32)>,
 'artist_genres_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"'dance pop', 'pop', 'r&b'"], dtype=object)>,
 'artist

### Train files

In [ ]:
train_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'data/{DATA_VERSION}/{TRAIN_DIR_PREFIX}/', delimiter="/"):
    if '.tfrecords' in blob.name:
        train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
train = tf.data.TFRecordDataset(train_files)

print(f"train size: {len(list(train))}")

train_parsed = train.map(FEATURE_MAP)

In [29]:
train_parsed

<MapDataset element_spec={'album_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_name_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'album_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_uri_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_genres_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_name_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_pop_pl': TensorSpec(shape=(5,), dtype=tf.float32, name=None), 'artist_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_uri_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artists_followers_pl': TensorSpec(shape=(5,), dtype=tf.float32, name

In [30]:
for x in train_parsed.batch(1).take(1):
    print(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Anything Goes'], dtype=object)>, 'album_name_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b'The Big Revival', b'Just As I Am', b"Here's To The Good Times",
        b'You Get What You Give', b'True Believers']], dtype=object)>, 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:5NG7WZaCZZ12M5LJm0JcVc'], dtype=object)>, 'album_uri_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b'spotify:album:2R5PAxygJ4YeRuSwoalKam',
        b'spotify:album:2TRJMLAU1nVzGA7Zpvkl7L',
        b'spotify:album:5MH765pytbQasmDxXArTah',
        b'spotify:album:0AXoQGOZDaYSaOo0qCTiCr',
        b'spotify:album:6cowf7fdb5dgKoglTekOK8']], dtype=object)>, 'artist_followers_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5135879.], dtype=float32)>, 'artist_genres_can': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"'contemporary country', 'country', 'country pop', 'coun

**Finished**